# Latent Dirichlet Allocation

In [0]:
import pandas as pd

data = pd.read_csv('data', sep=",", header=None)

data.columns = ['text']

data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


The data is a collection of emails that are not labelled. Let's try extract topics from them!

## Preprocessing 

👇 You're used to it by now... Clean up! Store the cleaned text in a new dataframe column "clean_text".

In [0]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

# Apply to all texts
data['clean_text'] = data.text.apply(clean)
data['clean_text'] = data['clean_text'].astype('str')

data.head()

,text,clean_text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,"['gld', 'cunixb', 'cc', 'columbia', 'edu', 'ga..."
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...,"['atterlep', 'vela', 'ac', 'oakland', 'edu', '..."
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...,"['miner', 'kuhub', 'cc', 'ukans', 'edu', 'subj..."
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...,"['atterlep', 'vela', 'ac', 'oakland', 'edu', '..."
4,From: vzhivov@superior.carleton.ca (Vladimir Z...,"['vzhivov', 'superior', 'carleton', 'ca', 'vla..."


## Latent Dirichlet Allocation model

👇 Train an LDA model to extract potential topics.

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(data['clean_text'])

lda_model = LatentDirichletAllocation(n_components=2)

lda_vectors = lda_model.fit_transform(data_vectorized)

## Visualize potential topics

👇 The function to print the words associated with the potential topics is already made for you. You just have to pass the correct arguments!

In [0]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('god', 1525.4717244523756), ('edu', 1047.605932768434), ('one', 848.9370428547662), ('would', 829.4675798446387), ('christian', 817.5282113636817), ('people', 713.361464554199), ('subject', 667.0440426167145), ('line', 633.2323257037933), ('jesus', 626.4930903583826), ('organization', 554.8729036993376)]
Topic 1:
[('edu', 1080.394067231515), ('team', 958.4945748621878), ('game', 950.9194105572662), ('line', 723.7676742961548), ('ca', 686.1889498513065), ('hockey', 649.4906082770664), ('subject', 639.9559573832356), ('organization', 617.127096300612), ('player', 529.4378357217474), ('play', 520.6741534748511)]


## Predict topic of new text

👇 You can now use your LDA model to predict the topic of a new text. First, use your vectorizer to vectorize the example. Then, use your LDA model to predict the topic of the vectorized example.

In [0]:
example = ["My team performed poorly last season. Their best player was out injured and only played one game"]

example_vectorized = vectorizer.transform(example)

lda_vectors = lda_model.transform(example_vectorized)

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])

topic 0 : 0.04430847661940636
topic 1 : 0.9556915233805936
